## Some calculus with poblano: the benefit of 'hidden' values and differentials

In [1]:
from poblano import PoblanoRing as Poblano
%display latex

#### First we make a Poblano object
This requires us to specify the variables on which functions will depend.

Here we simply have a two-dimensional space defined by variables `x` and `y`.

Notice how we can (optionally) provide a special LaTeX representation of the variables.

In [2]:
r = Poblano('demo', ['x', 'y'], {'y': '\\mathsf{Y}'})
x, y = r.vars

show(x)
show(y)

x

y

#### Now we can make some constants and functions

To gain the benefits of poblano, use the `fxn` method on the poblano object `r` to make constants and functions.

To make a constant, one simply provides a symbol to `fxn`, as follows for `a`.

This produces a `PoblanoExpression` object, which has its symbol used in expressions, a value hidden behind the symbol (to keep expressions simple), a dictionary useful for substitutions later on, and a total differential, which is zero for a constant.

With the `print` function we can quickly look at a `PoblanoExpression` and see if it is constant. We can also use the `is_constant()` method allows us to check too.

In [3]:
a = r.fxn('a')

show(a.symbol)
show(a.value)
show(a.dict)
show(a.total_diff)

print(a)
show(a.is_constant())

a

a

{a: a}

0

PoblanoExpr: ( a : a ( constant))


True

Next we'll make a function that depends on the variables.

Note that here the symbol and value of `f` are different, the total differential is automatically computed from the specified value, and that poblano identifies `f` as nonconstant.

In [4]:
f = r.fxn('f', x ** 2)

show(f.symbol)
show(f.value)
show(f.dict)
show(f.total_diff)

print(f)
show(f.is_constant())

f

x^2

{f: x^2}

2*dx*x

PoblanoExpr: ( f : {x}^{2} ( nonconstant))


False

#### Hiding nasty values and derivatives

A key capability of `PoblanoExpression` objects is that we can specify the total differential independently of the value, as below (in fact one can specify a differential without giving a value at all).

Here we make two Sage variables, `g_x` and `g_y`, which are the partial derivatives of `g` with respect to `x` and `y`.
We do this to avoid letting Sage go berserk computing the nasty derivatives of the value of `g`, which is a nontrivial expression.

In [5]:
dx = r.d(x)
dy = r.d(y)

var('g_x g_y')

g = r.fxn('g', value=exp(tanh(x + cos(y)) ** (a + x)), total_diff=g_x * dx + g_y * dy)

show(g.symbol)
show(g.value)
show(g.dict)
show(g.total_diff)

print(g)
show(g.is_constant())

g

e^(tanh(x + cos(y))^(a + x))

{g: e^(tanh(x + cos(y))^(a + x))}

dx*g_x + dy*g_y

PoblanoExpr: ( g : e^{\left(\tanh\left({x} + \cos\left({\mathsf{Y}}\right)\right)^{a + {x}}\right)} ( nonconstant))


False

Below we show that a `PoblanoExpression` is always represented by its symbol (in particular, note `g` in this case), which yields dramatically simpler results when one asks Sage to simplify an expression, which can easily produce unwieldly expressions.
This occurs below when we ask Sage to simplify the expression.
When the complicated value of `g` is 'hidden' behind the symbol, thanks to it being a `PoblanoExpression`, the expression is nice and simple.
On the other hand, when the full value of the expression is used, the expression becomes practically useless.

Next we'll demonstrate how nice it is to 'hide' nasty derivatives too.

In [6]:
show((f + a * exp(g ** 2)).simplify_full())
show((f + a * exp(g.value) ** 2).simplify_full())

a*e^(g^2) + f

a*e^(2*e^(((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x))) + f

### Time to calculate derivatives

To calculate derivatives of expressions involving `PoblanoExpression` objects, one should use the `diff` method of `r`.

In [7]:
show(r.diff(f, x))
show(r.diff(f, y))

show(r.diff(g, x))
show(r.diff(g, y))

2*x

0

g_x

g_y

We can build expressions of Sage and poblano expressions without special syntax (an exception is operators such as `cos(f)` or `exp(f)`, in which one must provide the symbol of a `PoblanoExpression`, e.g., `cos(f.symbol)`.).

For instance, `a * g + f` is a valid expression, and note that `g_x` and `g_y` show up in the derivative of `g`.

In [8]:
show(r.diff(a * g + f, x))
show(r.diff(a * g + f, y))

a*g_x + 2*x

a*g_y

We can go further with this, chaining arbitrary expressions.

In [9]:
show((r.diff(f / g + cos(a * f), x)).simplify_full())
show((r.diff(f / g + cos(a * f), y)).simplify_full())

-(f*g_x + 2*(a*g^2*sin(a*f) - g)*x)/g^2

-f*g_y/g^2

Compare this to the derivative if we use the value of `g` without hiding the differentials behind `g_x` and `g_y`.

The results obtained without 'hiding' the differentials are entirely unclear.

In [10]:
show((r.diff(f / g.value + cos(a * f), x)).simplify_full())
show((r.diff(f / g.value + cos(a * f), y)).simplify_full())

(2*a*x*cosh(x)*e^(((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x))*sin(a*f)*sinh(x) - 4*(a*x*cosh(x)*e^(((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x))*sin(a*f)*sinh(x) - x*cosh(x)*sinh(x))*cosh(cos(y))^2 - (2*f*cosh(x)*cosh(cos(y))^2*sinh(x) - f*cosh(x)*sinh(x) + (2*f*cosh(x)^2 - f)*cosh(cos(y))*sinh(cos(y)))*((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x)*log((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y)))) - 2*x*cosh(x)*sinh(x) + 2*(2*x*cosh(x)^2 - (2*a*x*cosh(x)^2*sin(a*f) - a*x*sin(a*f))*e^(((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x)) - x)*cosh(cos(y))*sinh(cos(y)) - (a*f + f*x)*((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x))/(2*cosh(x)*e^(((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x))*sinh(x)*sinh(cos(y))^2 + (2*sinh(x)^2 + 1)*cosh(cos(y))*e^(((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x))*sinh(cos(y)) + cosh(x)*e^(((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x))*sinh(x))

(a*f + f*x)*((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x)*sin(y)/(2*cosh(x)*e^(((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x))*sinh(x)*sinh(cos(y))^2 + (2*sinh(x)^2 + 1)*cosh(cos(y))*e^(((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x))*sinh(cos(y)) + cosh(x)*e^(((cosh(cos(y))*sinh(x) + cosh(x)*sinh(cos(y)))/(cosh(x)*cosh(cos(y)) + sinh(x)*sinh(cos(y))))^(a + x))*sinh(x))

If you now want to take the clear result with `g_x` and `g_y` and substitute the actual derivatives, simply substitute the differentiated _value_ of `g`:

In [11]:
show(r.diff(f / g + cos(a * f), x).subs(g_x=r.diff(g.value, x)))
show(r.diff(f / g + cos(a * f), y).subs(g_y=r.diff(g.value, y)))

-2*(a*sin(a*f) - 1/g)*x + f*tanh(x + cos(y))^(a + x)*((tanh(x + cos(y))^2 - 1)*(a + x)/tanh(x + cos(y)) - log(tanh(x + cos(y))))*e^(tanh(x + cos(y))^(a + x))/g^2

-(tanh(x + cos(y))^2 - 1)*(a + x)*f*tanh(x + cos(y))^(a + x - 1)*e^(tanh(x + cos(y))^(a + x))*sin(y)/g^2

Or better yet, simply print out the actual values of `g_x` and `g_y` next to the simple expressions:

In [12]:
show(r.diff(f / g + cos(a * f), x))
show(r.diff(f / g + cos(a * f), y))
show(g_x == r.diff(g.value, x))
show(g_y == r.diff(g.value, y))

-2*(a*sin(a*f) - 1/g)*x - f*g_x/g^2

-f*g_y/g^2

g_x == -tanh(x + cos(y))^(a + x)*((tanh(x + cos(y))^2 - 1)*(a + x)/tanh(x + cos(y)) - log(tanh(x + cos(y))))*e^(tanh(x + cos(y))^(a + x))

g_y == (tanh(x + cos(y))^2 - 1)*(a + x)*tanh(x + cos(y))^(a + x - 1)*e^(tanh(x + cos(y))^(a + x))*sin(y)